# vue-computed 属性和 watcher

### computed 属性

模板中使用表达式只适用于简单的操作。

在模板中放入太多的逻辑，会使模板过度膨胀和难以维护。

对于所有复杂逻辑，你都应该使用 computed 属性(computed property)。

### 基础示例

```html
<div id="example">
  <p>初始 message 是："{{ message }}"</p>
  <p>计算后的翻转 message 是："{{ reversedMessage }}"</p>
</div>
```

```js
var vm = new Vue({
  el: '#example',
  data: {
    message: 'Hello'
  },
  computed: {
    // 一个 computed 属性的 getter 函数
    reversedMessage: function () {
      // `this` 指向 vm 实例
      return this.message.split('').reverse().join('')
    }
  }
})
```

Vue 能够意识到 `vm.reversedMessage` 依赖于 `vm.message`，也会在 `vm.message` 修改后，更新所有依赖于 `vm.reversedMessage` 的数据绑定。

最恰到好处的部分是通过声明式来创建这种依赖关系：computed 属性的 getter 函数并无副作用(side effect)，因此也更加易于测试和理解。

### computed 缓存 vs method 方法

在表达式中通过调用 method 方法的方式，也能够实现与 computed 属性相同的结果：

```html
<p>翻转 message 是："{{ reverseMessage() }}"</p>
```

```js
// 在组件中
methods: {
  reverseMessage: function () {
    return this.message.split('').reverse().join('')
  }
}

```

这里不使用 computed 属性，而是在 methods 中定义一个相同的函数。对于最终结果，这两种方式确实恰好相同。

然而，细微的差异之处在于，computed 属性会基于它所依赖的数据进行缓存。每个 computed 属性，只有在它所依赖的数据发生变化时，才会重新取值(re-evaluate)。只要 message 没有发生变化，多次访问 computed 属性 reversedMessage，将会立刻返回之前计算过的结果，而不必每次都重新执行函数。

这也意味着，如下的 computed 属性永远不会更新，因为 `Date.now()` 不是一个响应式的依赖数据：
```js
computed: {
  now: function () {
    return Date.now()
  }
}
```

相比之下，每当触发重新渲染(re-render)时，method 调用方式将总是再次执行函数。

在某些场景下，你可能不希望有缓存，请使用 method 方法替代computed。

### computed 属性和 watch 属性

Vue 还提供了一种更加通用的方式，来观察和响应 Vue 实例上的数据变化：watch 属性。

当你有一些数据需要随着另外一些数据变化时，过度滥用 watch 属性会造成一些问题。

因此，更推荐的方式是，使用 computed 属性，而不是命令式(imperative)的 watch 回调函数。

### computed 属性中设置 setter

computed 属性默认只设置 getter 函数，不过在需要时，还可以提供 setter 函数：
```js
// ...
computed: {
  fullName: {
    // getter 函数
    get: function () {
      return this.firstName + ' ' + this.lastName
    },
    // setter 函数
    set: function (newValue) {
      var names = newValue.split(' ')
      this.firstName = names[0]
      this.lastName = names[names.length - 1]
    }
  }
}
// ...
```

现在当你运行 `vm.fullName = 'John Doe'`，将会调用 setter，然后会对应更新 `vm.firstName` 和 `vm.lastName`。

### watcher

在大多数情况下，更适合使用 computed 属性，然而有些时候，还是需要一个自定义 watcher。

这就是为什么 Vue 要通过 watch 选项，来提供一个更加通用的响应数据变化的方式。

当你需要在数据变化响应时，执行异步操作，或高性能消耗的操作，自定义 watcher 的方式就会很有帮助。

除了 watch 选项之外，还可以使用命令式(imperative)的 `vm.$watch` API。